In [1]:
## Visual Flow for DataShip Pipeline

# 1. Authenticate (Colab, Drive, Sheets)
        # ↓
# 2. Access Validation (Permissions, Owner Check)
        # ↓
# 3. Extract Raw Data (Sheets, Drive)
        # ↓
# 4. Clean Data (Missing values, duplicates, wrong formats)
        # ↓
# 5. Transform & Aggregate (KPI Calculations, Rollups, Metrics)
        # ↓
# 6. Output Final Data (Google Sheets, Looker Dashboard)
        # ↓
# 7. Automate Everything (Schedule / Notify / Refresh)

## Install Necessary Packages and Libraries

In [ ]:
!pip install emoji
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [27]:
# import Libraries

#import pandas as pd

import re
import os
import emoji
import hashlib
import openpyxl
import numpy as np

In [3]:
# Google Colab & Google API imports

''' These libraries are needed to authenticate with Google and interact with Google Sheets and Google Drive. '''

from google.colab import auth
from google.colab import drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.auth.transport.requests import Request
import gspread
from gspread.exceptions import SpreadsheetNotFound
from google.oauth2.credentials import Credentials
from google.auth import default
from gspread.exceptions import SpreadsheetNotFound, WorksheetNotFound


## Authentication for Google Services to Connect to Data Sources


→ This code is handling Google Authentication inside a Google Colab notebook to allow the user to access Google Drive folder for project (e.g. 12 - Dataship Client Success Reporting) and Google Sheets data (e.g. 00 - Mentee Data) programmatically.

This is common in projects like DataShip Client Success Reporting, where a lot of your data is stored in Google Sheets/Drive and you want to automate interaction with it.

In [4]:
# Authentication for Google Services setup (specifically Google Drive and Google Sheets)

# 1. Authenticate the User in Google Colab
auth.authenticate_user()

# 2. Get Google Credentials
import google.auth
creds, _ = google.auth.default()

# 3. Authorize Google Sheets API
gc = gspread.authorize(creds) #Interact with Google Sheets like a database (Read, Write, Update).


# Initialize the Drive API client
from google.colab import auth
from googleapiclient.discovery import build

#Initialize Google Drive API
service = build('drive', 'v3')


# Get authenticated user email properly in Colab
about = service.about().get(fields="user").execute()
user_email = about['user']['emailAddress']

print(f"Authenticated User Email: {user_email}")


Authenticated User Email: abubakaralfaki@gmail.com


## Checking folder permissions

In [5]:
def check_folder_access(service, folder_id, user_email):
    """
    Check folder owner, detailed permissions, and whether authenticated user has access.
    """

    try:
        # Get Folder Owner
        folder_metadata = service.files().get(fileId=folder_id, fields='owners').execute()
        owners = [owner['emailAddress'] for owner in folder_metadata.get('owners', [])]

        # Get Permissions
        permissions = service.permissions().list(fileId=folder_id, fields='permissions').execute()

        print("\nFolder Owner(s):")
        for owner_email in owners:
            print(owner_email)

        print("\nPermissions List (Type & Details):")
        users_with_access = []
        for permission in permissions.get('permissions', []):
            print(f"Type: {permission.get('type')} | Role: {permission.get('role')} | Email: {permission.get('emailAddress', 'N/A')}")
            # Grab emails only if it's a user
            if permission.get('type') == 'user' and 'emailAddress' in permission:
                users_with_access.append(permission['emailAddress'])

        # Check if user is Owner or Explicitly Shared
        if user_email in users_with_access or user_email in owners:
            print(f"\nAuthenticated user {user_email} has direct access to the folder.")
        else:
            print(f"\nAuthenticated user {user_email} might have indirect access (link/domain share), but is not directly listed.")

        return {'owners': owners, 'users_with_access': users_with_access, 'permissions': permissions.get('permissions', [])}

    except Exception as e:
        print(f"An error occurred: {e}")
        return {}


In [6]:
folder_id = '1r2EiMWNFVpV9WNsNwvu79WRuAHmzxKNs'

# Run acceess Check
users = check_folder_access(service, folder_id, user_email)


Folder Owner(s):
jaretandre1869@gmail.com

Permissions List (Type & Details):
Type: user | Role: writer | Email: imrelor@gmail.com
Type: user | Role: commenter | Email: daniel.r.marino1@gmail.com
Type: user | Role: commenter | Email: stern.elliot@gmail.com
Type: user | Role: writer | Email: alpha@avebagroup.com
Type: user | Role: writer | Email: vaibhavjain38@gmail.com
Type: user | Role: writer | Email: HanadSharmarke@gmail.com
Type: user | Role: writer | Email: jda.digital.corp.ltd@gmail.com
Type: user | Role: writer | Email: oluwatoke50@gmail.com
Type: user | Role: commenter | Email: amnaa.msvirtuals@gmail.com
Type: user | Role: commenter | Email: linetbaraka@gmail.com
Type: user | Role: writer | Email: jaretdandre@gmail.com
Type: user | Role: writer | Email: thealphadigitalph@gmail.com
Type: user | Role: commenter | Email: kushluthra15@gmail.com
Type: user | Role: writer | Email: abubakaralfaki@gmail.com
Type: user | Role: owner | Email: Jaretandre1869@gmail.com
Type: user | Role: 

# Extract Google Sheet data (00 - Mentee Data)

In [7]:
def replicate_google_sheet(gc, drive_service, original_sheet_id, new_sheet_name, target_folder_id):
    """
    Replicate a Google Sheet from a source to a clean working copy.
    """

    try:
        # Check if target sheet exists
        file = drive_service.files().get(fileId=gc.open(new_sheet_name).id, fields='trashed').execute()
        if file.get('trashed'):
            print(f"The sheet '{new_sheet_name}' is in the trash. Creating a new one.")
            new_sheet = gc.create(new_sheet_name)

        else:
            new_sheet = gc.open(new_sheet_name)
            print(f"The sheet '{new_sheet_name}' already exists. Updating its content.")

    except SpreadsheetNotFound:
        new_sheet = gc.create(new_sheet_name)
        print(f"The sheet '{new_sheet_name}' has been created.")

    # Open original sheet
    original_sheet = gc.open_by_key(original_sheet_id)

    for ws in original_sheet.worksheets():
        try:
            existing_ws = new_sheet.worksheet(ws.title)
            print(f"Worksheet '{ws.title}' found in '{new_sheet_name}'. Updating its content.")
            existing_ws.clear()
        except WorksheetNotFound:
            existing_ws = new_sheet.add_worksheet(title=ws.title, rows=ws.row_count, cols=ws.col_count)
            print(f"Worksheet '{ws.title}' created in '{new_sheet_name}'.")

        # Update values properly to avoid DeprecationWarning
        values = ws.get_all_values()
        existing_ws.update(range_name='A1', values=values)

    # Move to target folder
    file_metadata = drive_service.files().get(fileId=new_sheet.id, fields='parents').execute()
    current_parents = ",".join(file_metadata.get('parents', []))

    drive_service.files().update(
        fileId=new_sheet.id,
        addParents=target_folder_id,
        removeParents=current_parents
    ).execute()

    print(f"The sheet '{new_sheet_name}' is up-to-date and in the specified folder.")

    # Delete 'Sheet1' if it exists
    try:
        sheet1 = new_sheet.worksheet('Sheet1')
        new_sheet.del_worksheet(sheet1)
        print("Sheet1 deleted successfully.")
    except WorksheetNotFound:
        print("Sheet1 not found. No changes made.")


In [8]:
replicate_google_sheet(gc, service,
                       original_sheet_id='1hisTEqOD4VBJFB06kxnAHcCU02DuQ5moPcARELcTxV8',
                       new_sheet_name='mentee_data_copy',
                       target_folder_id='1QEDqJYty4_ZAPBRGrhYbdODXHTPYcO34')

The sheet 'mentee_data_copy' already exists. Updating its content.
Worksheet 'Overview' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Client Overview' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Client Job Search Data' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Booked Calls' found in 'mentee_data_copy'. Updating its content.
Worksheet 'To deleteMentor Booked Calls' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Client Module Progress' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Module Feedback' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Calendly Feedback' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Call Feedback' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Pulse Check Experiment' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Graduation Survey' found in 'mentee_data_copy'. Updating its content.
Worksheet 'Cancellation Survery' found

# Convert worksheets to dataframe

In [19]:
import gspread
import pandas as pd

# Authenticate and open the Google Sheet by its key (assuming `gc` is your gspread client)

file_id = '1R2dRIv1YjLzpi43Tg3Bis7_SiQozsj3ih25XwMWiWZE' # Change file id to your dataset
new_sheet = gc.open_by_key(file_id)

# List of sheet names you want to target
target_sheets = ['Client Job Search Data' , 'Client Module Progress', 'Module Feedback' , 'Calendly Feedback' , 'Mentors' , 'Timeline Prediction' ]  # Replace with your sheet names

# Create a dictionary to store the dataframes
df_dict = {}

# Iterate through each worksheet and convert to a DataFrame
for worksheet in new_sheet.worksheets():

    # Check if the worksheet's title is in the target_sheets list
    if worksheet.title in target_sheets:
        # Get all records from the worksheet
        data = worksheet.get_all_values()

        # Convert the data into a pandas DataFrame, setting the second row as the header
        df_dict[worksheet.title] = pd.DataFrame(data[1:], columns=data[0])

In [10]:
import os
from google.colab import drive
drive.mount('/content/drive')

output_folder = '/content/drive/My Drive/12 - DataShip Client Success Reporting/01 - Development/temp_files'

# Make sure folder exists
os.makedirs(output_folder, exist_ok=True)

#save all sheets as csv
for sheet_name, df in df_dict.items():
  clean_name = sheet_name.replace(" ", "_")
  df.to_csv(f'{output_folder}/{clean_name}.csv', index=False)
  print(f"Saved {clean_name}.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved Client_Job_Search_Data.csv
Saved Client_Module_Progress.csv
Saved Module_Feedback.csv
Saved Calendly_Feedback.csv
Saved Mentors.csv
Saved Timeline_Prediction.csv


In [13]:
print(os.listdir(output_folder))

['Client_Module_Progress.csv)', 'Mentors.csv)', 'Module_Feedback.csv', 'Calendly_Feedback.csv)', 'Client_Job_Search_Data.csv)', 'Module_Feedback.csv)', 'Timeline_Prediction.csv', 'Calendly_Feedback.csv', 'Client_Job_Search_Data.csv', 'Mentors.csv', 'Client_Module_Progress.csv', 'Timeline_Prediction.csv)']


# Cleaning Stage

In [28]:
# function to reformat a certain sheet
# def nps_sheet_cleanup(df):
#     # Check if DataFrame is empty
#     if df.empty:
#         # Return an empty DataFrame with the desired columns if it's empty
#         return pd.DataFrame(columns=['Statistic', 'Statistic Value'] + df.columns[2:].tolist())

#     # Get the original column names
#     original_columns = df.columns.tolist()
#     # Create the new data by combining the original columns and data values
#     new_data = [original_columns] + df.values.tolist()

#     # Instead of defining a fixed number of new columns,
#     # use the original columns plus the new 'Statistic' and 'Statistic Value' columns
#     new_columns = ['Statistic', 'Statistic Value'] + original_columns[2:]

#     # Create the new DataFrame with the adjusted columns
#     new_df = pd.DataFrame(new_data, columns=new_columns)

#     # Drop the unnecessary 'DELETE' column if it exists
#     if 'DELETE' in new_df.columns:
#         new_df = new_df.drop(columns='DELETE')

#     # Create module columns dynamically based on existing columns
#     module_columns = [col for col in new_df.columns if col.startswith('Module')]

#     # Set module column values to NaN if they don't already exist
#     for col in module_columns:
#         if col not in new_df.columns:
#             new_df[col] = np.nan

#     return new_df

# making everything lowercase for future text analysis
def df_lowercase (df):
    # Convert column names to lowercase if they are strings
    df.columns = [col.lower() if isinstance(col, str) else col for col in df.columns]

    # Apply lowercase conversion only to string values in the DataFrame
    df = df.map(lambda x: x.lower() if isinstance(x, str) else x)
    return df

# removing extra whitespace in column names
def remove_extra_col_ws(df):
    # Convert column names to strings before applying strip()
    df.columns = [re.sub(r'\s+', ' ', str(col).strip()) for col in df.columns]
    return df

# removing extra whitespace in values
def remove_extra_row_ws(value):
    if isinstance(value, str): # Ensure the text is a string
        return re.sub(r'\s+', ' ', value.strip())
    return value

# removing emojis
def remove_emojis(text):
    if isinstance(text, str):  # Ensure the text is a string
        return emoji.replace_emoji(text, replace='')
    return text

# Check if 'answer/feeback' is in the dataframe and replace
def calendly_sheet_cleanup(df):
    # Ensure the column exists in the DataFrame
    if 'answer/feedback' in df.columns:
        # Define the mapping for replacements
        replacements = {f"{i} - ": i for i in range(11)}  # "10 - " to "0 - "
        replacements.update({str(i): i for i in range(11)})  # "10" to "0"

        # Perform the replacements
        df['answer/feedback'] = df['answer/feedback'].replace(replacements)

    return df


def module_feedback_cleanup(df):
    # Ensure the column exists in the DataFrame
    if 'on a scale of 0 to 10, how likely are you to recommend dataship to a friend/colleague?' in df.columns:
        # Define the mapping for replacements
        replacements = {f"{i} - ": i for i in range(11)}  # "10 - " to "0 - "
        replacements.update({str(i): i for i in range(11)})  # "10" to "0"

        # Perform the replacements
        df['on a scale of 0 to 10, how likely are you to recommend dataship to a friend/colleague?'] = df['on a scale of 0 to 10, how likely are you to recommend dataship to a friend/colleague?'].replace(replacements)

    return df

## 1. Running the Cleaning Process

In [29]:
# Step 1

# Step 2
cleaned_df_dict = {}

# # Only apply nps_sheet_cleanup to the NPS sheet
# reference_df = df_dict['NPS']

# First, rename 'Cancellation Survery' to 'Cancellation Survey' if it exists
if 'Cancellation Survery' in df_dict:
    df_dict['Cancellation Survey'] = df_dict.pop('Cancellation Survery')

# Now apply the same cleaning logic to each sheet
for sheet_name in df_dict.keys():
    current_df = df_dict[sheet_name]
    cleaned_df_dict[sheet_name] = current_df

    # if current_df.columns.tolist() == reference_df.columns.tolist():
        # current_df = nps_sheet_cleanup(current_df)
        # Changing email address column to just email
    # col_name = ['email addess']
    # for column_name in current_df.columns:
    #     if column_name in col_name:
    #       current_df.rename(columns={'email address': 'email'}, inplace=True)

    cleaned_df_dict[sheet_name] = current_df
    current_df = current_df.reset_index(drop=True)
    current_df = df_lowercase(current_df)  # Applying lowercase here
    current_df = remove_extra_col_ws(current_df)
    current_df = current_df.map(remove_extra_row_ws)
    current_df = current_df.map(remove_emojis)
    current_df = current_df.map(lambda x: x.replace('"', '').replace("'", '') if isinstance(x, str) else x)
    current_df = module_feedback_cleanup(current_df)
    current_df = calendly_sheet_cleanup(current_df)
    cleaned_df_dict[sheet_name] = current_df



# Step 3
# Down below are the specific fixes such as type conversion, modifying certain sections and replacing cells

# List of columns that should be converted to dates
# List of columns to be converted to datetime

# Assuming 'current_df' is your DataFrame and 'sheet_name' is defined elsewhere
    date_columns = ['start date', 'end date', 'timestamp', 'date']

    for column_name in current_df.columns:
        if column_name in date_columns:
            # Step 1: Handle date-time format inconsistency and add a default time for date-only entries
            current_df[column_name] = current_df[column_name].apply(
                lambda x: x if isinstance(x, str) and len(x.split()) > 1 else f'{x} 00:00:00'
            )

            # Step 2: Convert the column to datetime, allowing various formats (including time)
            current_df[column_name] = pd.to_datetime(current_df[column_name], errors='coerce', dayfirst=False)

            # Print for the 'Graduation Survey' sheet
            if sheet_name == 'Graduation Survey':
                print(f"After datetime conversion for {column_name}:")
                print(current_df[column_name])

            # Step 3: Eliminate the time part (convert to date only)
            current_df[column_name] = current_df[column_name].dt.date

            # Print for the 'Graduation Survey' sheet
            if sheet_name == 'Graduation Survey':
                print(f"After eliminating time for {column_name}:")
                print(current_df[column_name])

    # Step 4: Convert datetime columns to strings in 'YYYY-MM-DD' format before exporting to JSON
    for column_name in current_df.columns:
        if column_name in date_columns:
            # Ensure the datetime is valid before formatting
            current_df[column_name] = current_df[column_name].apply(
                lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else None
            )

            # Print for the 'Graduation Survey' sheet
            if sheet_name == 'Graduation Survey':
                print(f"After formatting for {column_name}:")
                print(current_df[column_name])



    columns_to_combine = ['first name', 'last name']
    for column_name in current_df.columns:
        if column_name in columns_to_combine:
            current_df['name'] = current_df['first name'] + ' ' + current_df['last name']

    # renaming module names
    module_columns = ['module', 'what module is this call related to, if unsure choose the max module you are on right now']
    # renaming module names
    for column_name in current_df.columns:
        if column_name in module_columns:
            # Update each cell in the column
            current_df[column_name] = current_df[column_name].apply(lambda x: f'module {x}' if pd.notna(x) and not isinstance(x, int) else x)

    # Replacing values to NaNs
    col_name = ['linkedin assessments', 'did you find a better alternative to our service? if yes, what influenced your decision to switch?', 'module']
    values = ['[]', 'n?a', 'module 7', 'module 8', 'module all', 'module prep']
    for value in current_df.columns:
        # Check if the column name is in the col_name list
        if value in col_name:
            # Replace 'module 9' with 'module graduation'
            current_df[value] = current_df[value].replace('module 9', 'module graduation')
            # Replace the other values in the list with NaN
            current_df[value] = current_df[value].replace(values, np.nan)

    # Dropping rows by name - only if 'name' column exists
    names_to_drop = ['jaret andre']
    if 'name' in current_df.columns:  # Check if 'name' column exists
        current_df = current_df[~current_df['name'].isin(names_to_drop)]

    # replacing longer strings
    col_name = ['have you finished module including the worksheet(s)?', 'name', 'email']
    replacements = {'yes - i have or i will finish the module & worksheets before someone reviews it': 'yes',
                'no - i will not finish the module & worksheets before someone reviews it': 'no',
                '( syed ) atif': 'atif',
                'pagnihotri@iitbhilai.ac.in': 'prash.agnihotri@gmail.com', # special case
                'sreedev raja rajeswari': 'sreedev rajeswari',
                'samrawit girma megeressa': 'samrawit megeressa',
                'samrawit megerssa': 'samrawit megeressa',
                'quazi rasool':'quazi ziaur rasool',
                'elena':'elena tarasova',
                'dhairya purneshkumar dave':'dhairya dave',
                'ann george': 'ann maria chirackal george' # special case Calendly Feedback no email
                    }
    for column_name in current_df.columns:
        if column_name in col_name:
            current_df[column_name] = current_df[column_name].replace(replacements)

    # Changing email address column to just email
    col_name = ['email addess']
    for column_name in current_df.columns:
        if column_name in col_name:
          current_df.rename(columns={'email address': 'email'}, inplace=True)
    # cleaned_df_dict[sheet_name] = current_df

    current_df.drop_duplicates(inplace=True)
    cleaned_df_dict[sheet_name] = current_df


if 'Cancellation Survery' in cleaned_df_dict:
    del cleaned_df_dict['Cancellation Survery']

<ipython-input-29-39a17a9d8a69>:56: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  current_df[column_name] = pd.to_datetime(current_df[column_name], errors='coerce', dayfirst=False)
<ipython-input-29-39a17a9d8a69>:56: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  current_df[column_name] = pd.to_datetime(current_df[column_name], errors='coerce', dayfirst=False)
<ipython-input-29-39a17a9d8a69>:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df[column_name] = curre

In [32]:
print(cleaned_df_dict.items())

dict_items([('Client Job Search Data',                   name    status                          email  \
0        abhigna pilli  inactive       abhignapilli88@gmail.com   
1     abubakar al-faki    active       abubakaralfaki@gmail.com   
2      alexander woods    active         alexwoods309@gmail.com   
3    alexandria ransom  inactive  alexandriaransom880@gmail.com   
4      ali hussein ali  inactive          alihali1023@gmail.com   
..                 ...       ...                            ...   
100             vimala  inactive                                  
101            william  inactive                                  
102      xuening chang    active     amychang.datasci@gmail.com   
103       yasin sharaf    active           ysharaf998@gmail.com   
104    zachary wilkins  inactive               z_wilk@yahoo.com   

              trello board id  start date    end date total months paused  \
0                              2022-03-30        None                       
1 